In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import warnings

In [2]:
from yml2api import yml2api
from pathlib import Path

In [3]:
p = Path("./embedding_schema.yml")

In [60]:
g = (yml2api(p))

In [68]:
print(next(g))

StopIteration: 

In [3]:
y = yaml.load("embedding_schema.yml")

In [254]:
with open("embedding_schema.yml", 'r') as stream:
    y=yaml.load(stream)

In [33]:
y["Texts"]["properties"]

{'texts': {'type': 'Generator[string]', 'description': 'Texts generator'},
 'meta': {'type': 'Generator[dict]', 'description': 'Meta generator'}}

In [262]:
"""
from persistable import Persistable
from persistable.util.dict import merge_dicts

"""

'\nfrom persistable import Persistable\nfrom persistable.util.dict import merge_dicts\n\n'

In [297]:
objects = (
f'''
class {object_name}(Persistable):
    """
    {obj["description"]}

    Payload
    -------
    payload     : recdefaultdict
    |-- [key1]      : [type1]
    |   Description for [key1]
    |-- [key2]      : [type2]
    |   Description for [key1]
    |   |-- [key2.1] : [type2.1]
    |   |   Description for [key1]

    Properties
    ----------
''' + "\n".join(
    [
f'''
    {obj_property} : {obj['properties'][obj_property]['type']}
        {obj['properties'][obj_property]['description']}
''' for obj_property in obj["properties"]
    ] if "properties" in obj else ["None"]
) +
f'''

    Dependencies
    ------------
''' + "\n".join(
    [
        f"    {dep} : {obj['dependencies'][dep]['type']}" +
        "\n        {obj['dependencies'][dep]['description']}"
        for dep in obj["dependencies"]
    ] if "dependencies" in obj else ["None"]) +
f'''
           
    def __init__(params, intermediate_datapath, verbose=True):
       """
       Initiate persistable object.
    
       Parameters
       ----------
       params : dict
    
           Required Parameters
           -------------------
''' + "\n".join(
    [
        f"            {req_params} : {obj['required_parameters'][req_params]['type']}" +
        f"\n                {obj['required_parameters'][req_params]['description']}"
        for req_params in obj["required_parameters"]
    ] if "required_parameters" in obj else ["None"]) +
f'''
            
            Optional Parameters
            -------------------
''' + "\n".join(
    [
        f"            {opt_param}     : {obj['optional_parameters'][opt_param]['type']}" +
        f"\n                {obj['optional_parameters'][opt_param]['description']}"
        for opt_param in obj["optional_parameters"]
    ] if "optional_parameters" in obj else ["None"]) +
f'''
           
        intermediate_datapath : Path
           The path where intermediate data is persisted / loaded
        verbose : bool
           Verbosity flag (False dumps only WARNING level logs)
        """
        
        super().__init__(
           payload_name={repr(obj["name"])},
           params=merge_dicts(
               dict(
''' + "\n".join(
    [
        f"                    {opt_param}={repr(obj['optional_parameters'][opt_param].get('default', None))}"
        for opt_param in obj["optional_parameters"]
    ] if "optional_parameters" in obj else []) +
f'''
               ),
               params
           ),
           workingdatapath=intermediate_datapath,
           required_params={tuple(req_param for req_param in obj["required_parameters"])},
           excluded_fn_params=[],
           verbose=verbose
       )
    
    def _generate_payload(self, **untracked_payload_params):
       """
       Function that defines how payload is generated.
    
       Parameters
       ----------
       untracked_payload_params    : dict
       """
       pass
           
''' + "\n".join(
[
f'''
    @property
    def {obj_property}(self):
        """
        {obj["properties"][obj_property]["description"]}
        """

        raise NotImplementedError("Property not implemented.")

''' for obj_property in obj["properties"]
] if "properties" in obj else [])
for object_name, obj in y.items())

In [298]:
print(next(objects))


class Texts(Persistable):
    """
    Generic texts object that generates a text-string and meta data record for each document.

    Payload
    -------
    payload     : recdefaultdict
    |-- [key1]      : [type1]
    |   Description for [key1]
    |-- [key2]      : [type2]
    |   Description for [key1]
    |   |-- [key2.1] : [type2.1]
    |   |   Description for [key1]

    Properties
    ----------

    texts : Generator[string]
        Texts generator


    meta : Generator[dict]
        Meta generator


    Dependencies
    ------------
None
           
    def __init__(params, intermediate_datapath, verbose=True):
       """
       Initiate persistable object.
    
       Parameters
       ----------
       params : dict
    
           Required Parameters
           -------------------
            domain : string
                Select cloud or local
            datafileslocator : string
                Universal locator such as a filepath or bucket uri.  It is best practice 